In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import sys
import numpy
import pandas
import matplotlib
import seaborn
import scipy

print('Python: {}'.format(sys.version))
print('Numpy: {}.format(numpy.__version__)')
print('Pandas: {}.format(pandas.__version__)')
print('Matplotlib: {}.format(matplotlib.__version__)')
print('Seaborn: {}.format(seaborn.__version__)')
print('Scipy: {}.format(scipy.__version__)')
print('Sklearn: {}.format(sklearn.__version__)')

In [ ]:
#import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Load the dataset from the csv file using pandas
import pandas as pd
data = pd.read_csv("../input/creditcardfraud/creditcard.csv")

In [ ]:
#explore the dataset
print(data.columns)#print the name of the columns

In [ ]:
print(data.shape)#print the columns and rows

In [ ]:
print(data.describe())#print the mean,min,max...etc for each column

In [ ]:
#data=data.sample(frac=0.1,random_state=1)
#used to fraction out a sample of the given data to save time and computational power

In [ ]:
#Plot a histogram of each parameter
data.hist(figsize=(20,20))
plt.show()

In [ ]:
#Determine number of fraud cases in the dataset
Fraud=data[data['Class']==1]#Taking out all the fraudulant cases from the dataset
Valid=data[data['Class']==0]#Taking out all the Valid cases from the dataset

outlier_fraction = len(Fraud)/ float(len(Valid))#Checking the ratio of fraudulant cases to valid cases
print(outlier_fraction)

print('Fraud Cases: {}'.format(len(Fraud)))
print('Valid Cases: {}'.format(len(Valid)))

In [ ]:
#Correlation matrix
cormat=data.corr()#shows us which of the features are important for the analysis
fig=plt.figure(figsize = (12,9))

sns.heatmap(cormat,vmax=0.8,square=True)#Making a Heatmap
plt.show()

In [ ]:
#Get all the columns from the Dataframe
columns = data.columns.tolist()

#Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["Class"]]

#Store the variable we'll be predicting on
target = "Class"

X = data[columns]#Dataset except Class
Y = data[target]#Only class

#Print the shapes of X and Y
print(X.shape)
print(Y.shape)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

#define a random state
state = 1

#define the outlier detection methods
classifiers = {
    "Isolation Forst" : IsolationForest(max_samples=len(X),contamination = outlier_fraction,random_state=state),
    
    "Local Outlier Factor" : LocalOutlierFactor(n_neighbors = 20,contamination = outlier_fraction)
}

In [ ]:
#Fit the model

plt.figure(figsize=(9, 7))
n_outliers = len(Fraud)


for i, (clf_name, clf) in enumerate(classifiers.items()):
    
    # fit the data and tag outliers
    if clf_name == "Local Outlier Factor":
        y_pred = clf.fit_predict(X)
        scores_pred = clf.negative_outlier_factor_
    else:
        clf.fit(X)#Normal fit for Isolation Forest
        scores_pred = clf.decision_function(X)
        y_pred = clf.predict(X)#Later predict for Isolation Forest
    
    # Reshape the prediction values to 0 for valid, 1 for fraud. 
    y_pred[y_pred == 1] = 0 #Valid
    y_pred[y_pred == -1] = 1 #Outliers are Fraud
    
    n_errors = (y_pred != Y).sum()
    
    # Run classification metrics
    print('{}: {}'.format(clf_name, n_errors))#print out the name and the number of errors
    print(accuracy_score(Y, y_pred))#accuracy score
    print(classification_report(Y, y_pred)) #comparing Y to y_pred